In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

repo_root = Path().resolve().parents[2]
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from modules._import_helper import safe_import_from
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix
from sklearn.calibration import calibration_curve

set_seed = safe_import_from('00_repo_standards.src.mlphys_core.seeding', 'set_seed')
load_data, get_feature_columns, split_data = safe_import_from(
    '03_ml_tabular_foundations.src.data',
    'load_data', 'get_feature_columns', 'split_data'
)
create_lightgbm_pipeline, calibrate_model = safe_import_from(
    '03_ml_tabular_foundations.src.models',
    'create_lightgbm_pipeline', 'calibrate_model'
)
expected_calibration_error, compute_permutation_importance = safe_import_from(
    '03_ml_tabular_foundations.src.eval',
    'expected_calibration_error', 'compute_permutation_importance'
)

set_seed(42)
reports_dir = Path("../reports")
reports_dir.mkdir(exist_ok=True)

print("✅ Setup complete")

## 1. Why Calibration Matters

**Problem**: Raw classifier outputs are **scores**, not probabilities.

**Example**: Model outputs 0.7 for 100 samples:
- **Well-calibrated**: ~70 are actually positive
- **Poorly-calibrated**: Only 50 are positive (overconfident)

**When calibration is critical**:
- Medical diagnosis (need reliable risk estimates)
- Cost-sensitive decisions (FP/FN tradeoffs)
- Ensembling (combining models by probability)
- User-facing systems (displaying confidence)

In [ ]:
# Load and split data
df = load_data()
feature_cols = get_feature_columns(df)
X = df[feature_cols].values
y = df['is_signal'].values

X_train, X_val, X_test, y_train, y_val, y_test = split_data(
    X, y, test_size=0.2, val_size=0.2, random_state=42
)

print(f"Dataset: {X.shape[0]:,} samples, {X.shape[1]} features")

# Train uncalibrated model
print("\nTraining uncalibrated GBM...")
pipeline_uncalibrated = create_lightgbm_pipeline(
    n_estimators=500, learning_rate=0.05, max_depth=7, 
    num_leaves=63, random_state=42
)
pipeline_uncalibrated.fit(X_train, y_train)

y_pred_uncal = pipeline_uncalibrated.predict_proba(X_test)[:, 1]
print(f"✅ Uncalibrated model trained")

## 2. Reliability Diagram

**Reliability diagram**: Predicted probability vs. observed frequency.

**Interpretation**:
- **Diagonal line**: Perfect calibration (predicted = observed)
- **Above diagonal**: Underconfident (predicting lower than actual)
- **Below diagonal**: Overconfident (predicting higher than actual)

In [ ]:
# Compute reliability diagram
prob_true, prob_pred = calibration_curve(y_test, y_pred_uncal, n_bins=10, strategy='uniform')

fig, ax = plt.subplots(figsize=(8, 8))

# Plot reliability curve
ax.plot(prob_pred, prob_true, 'o-', linewidth=2.5, markersize=10, 
        label='Uncalibrated Model', color='coral')
ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfect Calibration', alpha=0.6)

ax.set_xlabel('Predicted Probability', fontsize=12)
ax.set_ylabel('Observed Frequency', fontsize=12)
ax.set_title('Reliability Diagram (Before Calibration)', fontsize=13, fontweight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.grid(alpha=0.3)
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig(reports_dir / '05_reliability_before.png', dpi=150, bbox_inches='tight')
plt.show()

# Compute Expected Calibration Error (ECE)
ece_uncal = expected_calibration_error(y_test, y_pred_uncal, n_bins=10)

print(f"📊 Calibration Analysis (Before):")
print(f"  Expected Calibration Error (ECE): {ece_uncal:.4f}")
if ece_uncal > 0.10:
    print("  ⚠️ Poor calibration (ECE > 0.10)")
elif ece_uncal > 0.05:
    print("  ⚠️ Moderate miscalibration (ECE > 0.05)")
else:
    print("  ✅ Well-calibrated (ECE < 0.05)")

print(f"\n💡 Interpretation:")
if prob_true[-1] < prob_pred[-1]:
    print("  • Model is OVERCONFIDENT (predicts higher than reality)")
else:
    print("  • Model is UNDERCONFIDENT (predicts lower than reality)")

## 3. Calibration Methods

**Common methods**:
1. **Platt Scaling**: Fit logistic regression on predictions
2. **Isotonic Regression**: Fit non-parametric monotonic function
3. **Temperature Scaling**: Divide logits by temperature parameter

**We'll use Platt scaling** (fast, works well for tree-based models).

In [ ]:
# Calibrate model using validation set
print("Calibrating model using Platt scaling...")
pipeline_calibrated = calibrate_model(
    pipeline_uncalibrated, X_val, y_val, method='platt'
)

y_pred_cal = pipeline_calibrated.predict_proba(X_test)[:, 1]
print("✅ Model calibrated")

# Compute reliability diagram for calibrated model
prob_true_cal, prob_pred_cal = calibration_curve(y_test, y_pred_cal, n_bins=10, strategy='uniform')
ece_cal = expected_calibration_error(y_test, y_pred_cal, n_bins=10)

print(f"\n📊 Calibration Analysis (After):")
print(f"  Expected Calibration Error (ECE): {ece_cal:.4f}")
print(f"  Improvement: {(ece_uncal - ece_cal):.4f} ({(ece_uncal - ece_cal)/ece_uncal*100:.1f}% reduction)")

# Plot both reliability diagrams
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Before calibration
ax1.plot(prob_pred, prob_true, 'o-', linewidth=2.5, markersize=10, color='coral', label='Uncalibrated')
ax1.plot([0, 1], [0, 1], 'k--', linewidth=2, alpha=0.6, label='Perfect')
ax1.set_xlabel('Predicted Probability', fontsize=11)
ax1.set_ylabel('Observed Frequency', fontsize=11)
ax1.set_title(f'Before Calibration (ECE={ece_uncal:.4f})', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)
ax1.set_xlim([0, 1])
ax1.set_ylim([0, 1])

# After calibration
ax2.plot(prob_pred_cal, prob_true_cal, 's-', linewidth=2.5, markersize=10, color='steelblue', label='Calibrated')
ax2.plot([0, 1], [0, 1], 'k--', linewidth=2, alpha=0.6, label='Perfect')
ax2.set_xlabel('Predicted Probability', fontsize=11)
ax2.set_ylabel('Observed Frequency', fontsize=11)
ax2.set_title(f'After Calibration (ECE={ece_cal:.4f})', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)
ax2.set_xlim([0, 1])
ax2.set_ylim([0, 1])

plt.tight_layout()
plt.savefig(reports_dir / '05_reliability_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# Check discrimination is preserved
auc_uncal = roc_auc_score(y_test, y_pred_uncal)
auc_cal = roc_auc_score(y_test, y_pred_cal)

print(f"\n🔍 Discrimination Check:")
print(f"  Uncalibrated AUC: {auc_uncal:.4f}")
print(f"  Calibrated AUC:   {auc_cal:.4f}")
print(f"  Change: {(auc_cal - auc_uncal):.4f}")
print("  ✅ Calibration preserves discrimination (AUC unchanged)")

## 4. Explainability: Permutation Importance

**Goal**: Understand which features drive predictions.

**Method**: Permutation importance
- Shuffle feature i
- Measure drop in performance
- Importance = drop in AUC

**Advantages over tree importance**:
- Model-agnostic (works for any model)
- Captures feature interactions
- Doesn't bias towards high-cardinality features

In [ ]:
# Compute permutation importance
print("Computing permutation importance (this may take ~30s)...")
perm_importance = compute_permutation_importance(
    pipeline_calibrated, X_test, y_test, n_repeats=10, random_state=42
)

# Sort by importance
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': perm_importance['importances_mean'],
    'Std': perm_importance['importances_std']
}).sort_values('Importance', ascending=False)

print("✅ Permutation importance computed")

# Plot top 10 features
fig, ax = plt.subplots(figsize=(10, 7))
top_n = 10
top_features = importance_df.head(top_n)

ax.barh(range(top_n), top_features['Importance'].values, 
        xerr=top_features['Std'].values, alpha=0.8,
        color='steelblue', edgecolor='black', linewidth=1.5, capsize=5)
ax.set_yticks(range(top_n))
ax.set_yticklabels(top_features['Feature'].values, fontsize=10)
ax.set_xlabel('Permutation Importance (Drop in AUC)', fontsize=12)
ax.set_title(f'Top {top_n} Features by Permutation Importance', fontsize=13, fontweight='bold')
ax.grid(alpha=0.3, axis='x')
ax.invert_yaxis()

plt.tight_layout()
plt.savefig(reports_dir / '05_permutation_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Top 5 Most Important Features:")
for idx, row in importance_df.head(5).iterrows():
    print(f"  {row['Feature']:<20} {row['Importance']:.4f} ± {row['Std']:.4f}")

print(f"\n💡 Interpretation:")
print(f"  • Top feature: {importance_df.iloc[0]['Feature']} (likely m_inv, Higgs mass peak)")
print(f"  • Error bars show stability across permutations")
print(f"  • Features with Importance ≈ 0 can be dropped")

## 5. Error Analysis: Confusion Matrix Breakdown

**Goal**: Understand where the model fails.

**Strategy**:
1. Compute confusion matrix
2. Analyze False Positives vs. False Negatives
3. Identify systematic patterns

In [ ]:
# Confusion matrix at optimal threshold (using default 0.5 for simplicity)
y_pred_binary = (y_pred_cal >= 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred_binary)

fig, ax = plt.subplots(figsize=(7, 6))
im = ax.imshow(cm, cmap='Blues', alpha=0.8)

# Add text annotations
labels = [['TN\n(Correct Background)', 'FP\n(Background → Signal)'],
          ['FN\n(Signal → Background)', 'TP\n(Correct Signal)']]

for i in range(2):
    for j in range(2):
        text = ax.text(j, i, f'{cm[i, j]:,}\n{labels[i][j]}',
                      ha='center', va='center', fontsize=11, fontweight='bold',
                      color='white' if cm[i, j] > cm.max()/2 else 'black')

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['Pred: Background (0)', 'Pred: Signal (1)'], fontsize=11)
ax.set_yticklabels(['True: Background (0)', 'True: Signal (1)'], fontsize=11)
ax.set_title('Confusion Matrix (Calibrated Model, Threshold=0.5)', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig(reports_dir / '05_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Error analysis
tn, fp, fn, tp = cm.ravel()
total = cm.sum()

print(f"📊 Error Breakdown:")
print(f"  True Negatives (TN):  {tn:,} ({tn/total*100:.1f}%)")
print(f"  False Positives (FP): {fp:,} ({fp/total*100:.1f}%) ← Background misclassified as signal")
print(f"  False Negatives (FN): {fn:,} ({fn/total*100:.1f}%) ← Signal misclassified as background")
print(f"  True Positives (TP):  {tp:,} ({tp/total*100:.1f}%)")

print(f"\n🔍 Error Rates:")
print(f"  Precision: {tp/(tp+fp):.1%} (of predicted signals, {tp/(tp+fp):.1%} are real)")
print(f"  Recall:    {tp/(tp+fn):.1%} (of true signals, {tp/(tp+fn):.1%} were caught)")
print(f"  FPR:       {fp/(fp+tn):.1%} (of backgrounds, {fp/(fp+tn):.1%} misclassified)")
print(f"  FNR:       {fn/(fn+tp):.1%} (of signals, {fn/(fn+tp):.1%} missed)")

## 6. Slice Analysis: Performance by Subgroup

**Goal**: Find subgroups where model fails.

**Strategy**: Bin continuous features, measure performance per bin.

**Example**: Does model fail on low-momentum events? High-mass events?

In [ ]:
# Slice analysis: Performance by feature bins
slice_feature = 'm_inv'  # Invariant mass (most important feature)
slice_feature_idx = feature_cols.index(slice_feature)

# Create bins
n_bins = 5
bins = np.linspace(X_test[:, slice_feature_idx].min(), 
                   X_test[:, slice_feature_idx].max(), n_bins+1)
bin_labels = [f'{bins[i]:.1f}-{bins[i+1]:.1f}' for i in range(n_bins)]

# Assign samples to bins
bin_indices = np.digitize(X_test[:, slice_feature_idx], bins) - 1
bin_indices = np.clip(bin_indices, 0, n_bins-1)

# Compute metrics per bin
slice_metrics = []
for i in range(n_bins):
    mask = bin_indices == i
    if mask.sum() > 10:  # Need enough samples
        slice_auc = roc_auc_score(y_test[mask], y_pred_cal[mask])
        slice_metrics.append({
            'Bin': bin_labels[i],
            'N_samples': mask.sum(),
            'Signal_rate': y_test[mask].mean(),
            'AUC': slice_auc
        })

slice_df = pd.DataFrame(slice_metrics)

# Plot slice performance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# AUC by bin
ax1.bar(range(len(slice_df)), slice_df['AUC'].values, alpha=0.8, 
        color='steelblue', edgecolor='black', linewidth=1.5)
ax1.axhline(auc_cal, color='red', linestyle='--', linewidth=2, label=f'Overall AUC={auc_cal:.3f}')
ax1.set_xticks(range(len(slice_df)))
ax1.set_xticklabels(slice_df['Bin'].values, rotation=45, ha='right')
ax1.set_ylabel('AUC-ROC', fontsize=12)
ax1.set_xlabel(f'{slice_feature} Bins (GeV)', fontsize=12)
ax1.set_title(f'Performance by {slice_feature} Range', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3, axis='y')
ax1.set_ylim([0.85, 1.0])

# Sample distribution
ax2.bar(range(len(slice_df)), slice_df['N_samples'].values, alpha=0.8,
        color='coral', edgecolor='black', linewidth=1.5)
ax2.set_xticks(range(len(slice_df)))
ax2.set_xticklabels(slice_df['Bin'].values, rotation=45, ha='right')
ax2.set_ylabel('Number of Samples', fontsize=12)
ax2.set_xlabel(f'{slice_feature} Bins (GeV)', fontsize=12)
ax2.set_title('Sample Distribution Across Bins', fontsize=13, fontweight='bold')
ax2.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(reports_dir / '05_slice_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"📊 Slice Analysis ({slice_feature}):")
print(slice_df.to_string(index=False))

# Identify worst-performing slice
worst_slice = slice_df.loc[slice_df['AUC'].idxmin()]
print(f"\n⚠️ Worst-performing slice:")
print(f"  Bin: {worst_slice['Bin']} GeV")
print(f"  AUC: {worst_slice['AUC']:.4f} (vs. overall {auc_cal:.4f})")
print(f"  Samples: {worst_slice['N_samples']}")
print(f"\n💡 Mitigation: Collect more data in this range or engineer features specific to this regime")

## 7. Top Mistakes: Inspecting Misclassified Examples

**Goal**: Understand why specific examples fail.

**Strategy**: Find hardest false positives and false negatives.

In [ ]:
# Find top mistakes
# False Positives: background with high predicted probability
fp_mask = (y_test == 0) & (y_pred_cal > 0.5)
fp_scores = y_pred_cal[fp_mask]
fp_indices = np.where(fp_mask)[0]

# Sort by confidence (most confident mistakes)
top_fp_idx = fp_indices[np.argsort(fp_scores)[-5:]]  # Top 5 most confident FPs

# False Negatives: signal with low predicted probability
fn_mask = (y_test == 1) & (y_pred_cal < 0.5)
fn_scores = y_pred_cal[fn_mask]
fn_indices = np.where(fn_mask)[0]

# Sort by confidence (most confident mistakes)
top_fn_idx = fn_indices[np.argsort(fn_scores)[:5]]  # Top 5 most confident FNs

print("🔍 Top Mistakes Analysis")
print("=" * 60)

print("\nTop 5 False Positives (Background predicted as Signal):")
print(f"{'Index':<8} {'Pred Prob':<12} {'Feature Summary'}")
print("-" * 60)
for idx in top_fp_idx:
    pred_prob = y_pred_cal[idx]
    # Show key feature values
    m_inv_val = X_test[idx, feature_cols.index('m_inv')]
    missing_et_val = X_test[idx, feature_cols.index('missing_E_T')]
    print(f"{idx:<8} {pred_prob:<12.4f} m_inv={m_inv_val:.1f}, missing_E_T={missing_et_val:.1f}")

print("\nTop 5 False Negatives (Signal predicted as Background):")
print(f"{'Index':<8} {'Pred Prob':<12} {'Feature Summary'}")
print("-" * 60)
for idx in top_fn_idx:
    pred_prob = y_pred_cal[idx]
    m_inv_val = X_test[idx, feature_cols.index('m_inv')]
    missing_et_val = X_test[idx, feature_cols.index('missing_E_T')]
    print(f"{idx:<8} {pred_prob:<12.4f} m_inv={m_inv_val:.1f}, missing_E_T={missing_et_val:.1f}")

print("\n💡 Patterns in Mistakes:")
print("  • False Positives: Background events that 'look like' signal")
print("    - May have high m_inv (near Higgs mass) by chance")
print("    - Difficult to distinguish from true signal")
print("  • False Negatives: Signal events that 'look like' background")
print("    - May have low m_inv (off-peak)")
print("    - Low missing E_T (leptons not well-isolated)")

## 8. Summary Report: Failure Modes & Mitigations

Synthesize error analysis into actionable insights.

In [ ]:
# Generate error analysis summary
error_summary = f"""
# Error Analysis Summary

## Model Performance
- **Test AUC**: {auc_cal:.4f}
- **Test PR-AUC**: {average_precision_score(y_test, y_pred_cal):.4f}
- **Calibration (ECE)**: {ece_cal:.4f} (improved from {ece_uncal:.4f})

## Error Breakdown
- **False Positives**: {fp:,} ({fp/total*100:.1f}%)
  - Background events misclassified as signal
  - **Cost**: Wasted detector readout time, follow-up analysis
  
- **False Negatives**: {fn:,} ({fn/total*100:.1f}%)
  - Signal events missed
  - **Cost**: Missed physics discoveries

## Identified Failure Modes

### 1. Off-Peak Signal Events (False Negatives)
- **Pattern**: Signal with m_inv far from 125 GeV
- **Frequency**: ~{fn/(fn+tp)*100:.0f}% of all signal
- **Mitigation**: 
  - Add feature: distance to Higgs mass peak
  - Collect more off-peak training data
  - Ensemble with model trained on off-peak region

### 2. Background with Signal-Like Features (False Positives)
- **Pattern**: Background with high m_inv, high missing E_T
- **Frequency**: ~{fp/(fp+tn)*100:.0f}% of all background
- **Mitigation**:
  - Add discriminating features (jet substructure, angular correlations)
  - Increase threshold for high-stakes decisions
  - Use ensemble disagreement as uncertainty signal

### 3. Low-Momentum Events (Slice Analysis)
- **Pattern**: Worse performance at {worst_slice['Bin']} GeV
- **Impact**: AUC drops to {worst_slice['AUC']:.4f} (vs. {auc_cal:.4f} overall)
- **Mitigation**:
  - Stratified sampling to balance momentum bins
  - Dedicated model for low-momentum regime
  - Feature engineering for kinematics at threshold

## Recommendations

1. **Production Deployment**:
   - Use calibrated model (ECE < 0.05)
   - Set threshold based on cost analysis (FP vs. FN tradeoff)
   - Monitor slice performance (retrain if drift detected)

2. **Model Improvements**:
   - Collect more data in failure slices
   - Engineer features for off-peak signal
   - Ensemble multiple models (momentum-binned)

3. **Monitoring**:
   - Track calibration drift (ECE over time)
   - Monitor slice performance separately
   - Alert if FN rate increases (missed discoveries)
"""

# Save summary
with open(reports_dir / '05_error_analysis_summary.md', 'w') as f:
    f.write(error_summary)

print("📋 Error Analysis Summary")
print("=" * 60)
print(error_summary)
print(f"\n✅ Summary saved to: {reports_dir / '05_error_analysis_summary.md'}")

## 9. Exercises

**Exercise 1**: Diagnose calibration issues

You train a model and get this reliability diagram:
- Bin 0-0.1: Predicted=0.05, Observed=0.02
- Bin 0.1-0.2: Predicted=0.15, Observed=0.10
- ...
- Bin 0.9-1.0: Predicted=0.95, Observed=0.70

What does this tell you? How would you fix it?

In [ ]:
# Your answer here:
# 


**Exercise 2**: Feature importance discrepancy

You compute two types of feature importance:
- **Tree importance**: `feature_A` = 0.3, `feature_B` = 0.1
- **Permutation importance**: `feature_A` = 0.05, `feature_B` = 0.25

Why do they disagree? Which should you trust?

In [ ]:
# Your answer here:
# 


**Exercise 3**: Design an error mitigation strategy

Your fraud detection model has:
- High FP rate on small transactions (<$10)
- High FN rate on new merchants (< 30 days old)

Propose 3 concrete mitigations (data, features, or modeling).

In [ ]:
# Your answer here:
# 


---
## Solutions

**Solution 1**: Diagnose calibration issues

**Analysis**: Model is **systematically overconfident**.

**Evidence**:
- At high confidence (0.9-1.0), predicts 95% but only 70% are positive
- Consistent pattern across bins: predicted > observed

**Root causes**:
1. **Tree-based models** (GBMs) tend to be overconfident
   - Split decisions are deterministic
   - Leaf predictions don't capture uncertainty
   
2. **Class imbalance** amplifies overconfidence
   - Rare positive class → high thresholds
   - Model "reaches" for positives

3. **Overfitting** in high-confidence region
   - Memorized training patterns
   - Doesn't generalize to test distribution

**Fixes**:

1. **Platt Scaling** (first choice):
   ```python
   from sklearn.calibration import CalibratedClassifierCV
   calibrated = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
   calibrated.fit(X_val, y_val)  # Use separate validation set
   ```

2. **Isotonic Regression** (if non-monotonic):
   ```python
   calibrated = CalibratedClassifierCV(model, method='isotonic', cv='prefit')
   ```

3. **Reduce overconfidence in training**:
   - Early stopping (prevent overfitting)
   - Increase regularization (reg_alpha, reg_lambda)
   - Use ensembles (average predictions → more conservative)

4. **Temperature scaling** (for neural networks):
   ```python
   # Find temperature T that minimizes log loss on validation set
   # Divide logits by T before softmax
   ```

**Verification**: ECE should drop from ~0.25 to <0.05 after calibration.

In [ ]:
# Solution 1 (code demonstration)
print("Calibration Diagnosis")
print("=" * 60)
print("\nSymptoms:")
print("  Predicted=0.95, Observed=0.70 ← OVERCONFIDENT")
print("  Pattern: Predicted > Observed across all bins")
print("\nRoot causes:")
print("  1. Tree-based models are naturally overconfident")
print("  2. Class imbalance amplifies issue")
print("  3. Overfitting in high-confidence region")
print("\nFixes (prioritized):")
print("  1. Platt scaling (logistic recalibration)")
print("  2. Isotonic regression (non-parametric)")
print("  3. Temperature scaling (for neural networks)")
print("  4. Reduce overfitting (early stopping, regularization)")
print("\nExpected result:")
print("  ECE: 0.25 → 0.03 (90% reduction)")
print("  Reliability curve aligns with diagonal")

**Solution 2**: Feature importance discrepancy

**Why they disagree**:

**Tree importance (Gain-based)**:
- Measures: Total reduction in loss when feature is used to split
- **Bias**: Favors high-cardinality features (more split opportunities)
- **Issue**: Doesn't capture feature interactions or test-time importance
- `feature_A` has high gain → used frequently in splits

**Permutation importance**:
- Measures: Drop in test performance when feature is shuffled
- **Unbiased**: Reflects true predictive power at test time
- **Issue**: Computationally expensive (need to rescore data)
- `feature_B` has high permutation importance → critical for test predictions

**Interpretation**:
```
Tree Importance:  A=0.3,  B=0.1
Perm Importance:  A=0.05, B=0.25

→ Feature A: Used frequently in training (many splits),
              but NOT critical for test predictions (low permutation drop)
              
→ Feature B: Used less in training (fewer splits),
              but CRITICAL for test predictions (high permutation drop)
```

**Why this happens**:
1. **Correlated features**: A is correlated with other features
   - Tree uses A for splits, but other features can substitute
   - When A is permuted, other features compensate
   
2. **Feature interactions**: B interacts with other features
   - Not used alone (low gain), but critical in combination
   - When B is permuted, interactions break → performance drops

**Which to trust**: **Permutation importance** ✅

**Use case**:
- **Feature selection**: Drop features with low permutation importance
- **Explainability**: Report permutation importance to stakeholders
- **Tree importance**: Use for debugging training (which features are used)

**Example**:
```python
# Feature A: Age (correlated with income)
# Feature B: Fraud flag (rare, but decisive when present)

# Tree uses Age frequently (continuous, many splits)
# But Fraud flag is more predictive (permutation test reveals)
```

In [ ]:
# Solution 2 (code demonstration)
print("Feature Importance Discrepancy")
print("=" * 60)
print("\nObservation:")
print("  Tree Importance:  A=0.3,  B=0.1")
print("  Perm Importance:  A=0.05, B=0.25")
print("\nExplanation:")
print("  Feature A:")
print("    • High tree importance (used in many splits)")
print("    • Low permutation importance (not critical at test time)")
print("    • Likely correlated with other features (redundant)")
print("  Feature B:")
print("    • Low tree importance (few splits)")
print("    • High permutation importance (critical for predictions)")
print("    • Participates in feature interactions")
print("\nWhich to trust: Permutation Importance ✅")
print("  → Reflects true test-time predictive power")
print("  → Use for feature selection and explainability")
print("\nTree importance is useful for:")
print("  • Debugging training (which features model uses)")
print("  • Understanding split decisions")

**Solution 3**: Error mitigation strategy

**Problem**:
- **High FP** on small transactions (<$10)
- **High FN** on new merchants (< 30 days old)

**Mitigation Strategies**:

### 1. Data Collection (Address Distribution Gaps)

**Small transactions**:
```python
# Problem: Imbalanced data (most fraud is high-value)
# Solution: Collect more small-value fraud examples

# Upsample small-value frauds in training
small_fraud_mask = (df['amount'] < 10) & (df['is_fraud'] == 1)
df_balanced = pd.concat([df, df[small_fraud_mask].sample(n=1000, replace=True)])

# Or: Use SMOTE (synthetic oversampling)
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy={1: target_count})
X_resampled, y_resampled = smote.fit_resample(X, y)
```

**New merchants**:
```python
# Problem: Cold start (no history for new merchants)
# Solution: Collect labeled data specifically for new merchants

# Prioritize manual review for new merchants
# Build dataset: 10K transactions from new merchants with labels
```

### 2. Feature Engineering (Domain-Specific Signals)

**Small transactions**:
```python
# Add features capturing "small transaction patterns"
df['is_small_transaction'] = (df['amount'] < 10).astype(int)
df['small_tx_velocity'] = df.groupby('user_id')['is_small_transaction'].transform('sum')  # How many small tx recently?
df['amount_deviation'] = np.abs(df['amount'] - df.groupby('user_id')['amount'].transform('mean'))  # Unusual amount?

# Interaction: Small amount + high velocity → suspicious
df['small_high_velocity'] = (df['is_small_transaction'] & (df['small_tx_velocity'] > 5)).astype(int)
```

**New merchants**:
```python
# Add features capturing "new merchant risk"
df['merchant_age_days'] = (df['timestamp'] - df['merchant_first_seen']).dt.days
df['merchant_tx_count'] = df.groupby('merchant_id')['transaction_id'].transform('count')
df['merchant_fraud_rate'] = df.groupby('merchant_id')['is_fraud'].transform('mean')  # Historical fraud rate

# Cold start: Use merchant category as proxy
df['merchant_category_risk'] = df['merchant_category'].map(category_risk_dict)
```

### 3. Modeling (Subgroup-Specific Models)

**Small transactions**:
```python
# Option A: Separate model for small transactions
model_small = train_model(df[df['amount'] < 10])
model_large = train_model(df[df['amount'] >= 10])

# Route transactions to appropriate model
def predict(tx):
    if tx['amount'] < 10:
        return model_small.predict(tx)
    else:
        return model_large.predict(tx)
```

```python
# Option B: Adjust threshold for small transactions
threshold_default = 0.5
threshold_small = 0.3  # Lower threshold → catch more fraud (accept higher FP)

if tx['amount'] < 10:
    threshold = threshold_small
else:
    threshold = threshold_default
```

**New merchants**:
```python
# Option A: Conservative threshold for new merchants
if merchant_age < 30:
    threshold = 0.2  # Low threshold → route more to manual review
    
# Option B: Require 2FA for new merchants
if merchant_age < 30 and fraud_score > 0.3:
    require_2fa(transaction)
```

```python
# Option C: Ensemble with heuristic
rule_based_score = check_new_merchant_rules(tx)  # Heuristics
ml_score = model.predict_proba(tx)[1]
final_score = 0.7 * ml_score + 0.3 * rule_based_score  # Blend
```

**Summary**:
1. **Data**: Upsample minority failure modes
2. **Features**: Engineer domain-specific signals
3. **Modeling**: Subgroup-specific models or thresholds

In [ ]:
# Solution 3 (code template)
print("Error Mitigation Strategy")
print("=" * 60)
print("\nProblem:")
print("  • High FP on small transactions (<$10)")
print("  • High FN on new merchants (< 30 days old)")
print("\nStrategy 1: Data Collection")
print("  • Upsample small-value frauds (SMOTE)")
print("  • Prioritize labeling for new merchants")
print("\nStrategy 2: Feature Engineering")
print("  Small transactions:")
print("    • Add: small_tx_velocity, amount_deviation")
print("    • Interaction: small_amount × high_velocity")
print("  New merchants:")
print("    • Add: merchant_age_days, merchant_tx_count")
print("    • Proxy: merchant_category_risk")
print("\nStrategy 3: Modeling")
print("  Small transactions:")
print("    • Option A: Separate model for <$10")
print("    • Option B: Lower threshold (0.5 → 0.3)")
print("  New merchants:")
print("    • Option A: Conservative threshold (0.5 → 0.2)")
print("    • Option B: Require 2FA for new merchants")
print("    • Option C: Ensemble ML + heuristics")

---

## ✅ Notebook Complete!

**What you learned**:
1. ✅ Calibrate probability predictions (Platt scaling, ECE)
2. ✅ Explain predictions with permutation importance
3. ✅ Perform slice analysis (identify subgroup failures)
4. ✅ Analyze top mistakes (FP/FN patterns)
5. ✅ Design error mitigation strategies

**Outputs saved**:
- `reports/05_reliability_before.png`
- `reports/05_reliability_comparison.png`
- `reports/05_permutation_importance.png`
- `reports/05_confusion_matrix.png`
- `reports/05_slice_analysis.png`
- `reports/05_error_analysis_summary.md`

**Key takeaways**:
- Calibration matters for decision-making (use Platt scaling)
- Permutation importance > tree importance for explainability
- Slice analysis reveals hidden failure modes
- Error analysis drives feature engineering and data collection

**Portfolio value**: This notebook demonstrates production ML thinking—not just metrics, but **actionable insights** for improvement.

---

## 🎓 Congratulations!

You've completed the full tabular ML curriculum:
1. ✅ EDA & data quality
2. ✅ Splitting, CV & leakage prevention
3. ✅ Baselines & metrics
4. ✅ GBM training & hyperparameter tuning
5. ✅ Calibration, explainability & error analysis

**Next steps**:
- Apply to your own datasets
- Extend with SHAP for instance-level explanations
- Deploy model with monitoring (MLflow, Evidently)